In [3]:
import parmed as pmd
import subprocess as sp
import numpy as np

from openeye.oechem import *
from openeye.oeomega import *
from openforcefield.typing.engines.smirnoff import *
from pdbfixer import PDBFixer

In [4]:
ben_smiles = '[O-]C(=O)c1ccccc1'
bcd_smiles = 'C([C@@H]1[C@@H]2[C@@H]([C@H]([C@H](O1)O[C@@H]3[C@H](O[C@@H]([C@@H]([C@H]3O)O)O[C@@H]4[C@H](O[C@@H]([C@@H]([C@H]4O)O)O[C@@H]5[C@H](O[C@@H]([C@@H]([C@H]5O)O)O[C@@H]6[C@H](O[C@@H]([C@@H]([C@H]6O)O)O[C@@H]7[C@H](O[C@@H]([C@@H]([C@H]7O)O)O[C@@H]8[C@H](O[C@H](O2)[C@@H]([C@H]8O)O)CO)CO)CO)CO)CO)CO)O)O)O'

In [5]:
mols = []
for smiles in [ben_smiles, bcd_smiles]:
    mol = oechem.OEMol()
    oechem.OESmilesToMol(mol, smiles)
    for atom in mol.GetAtoms():
        atom.SetPartialCharge(atom.GetFormalCharge())
    oechem.OEAddExplicitHydrogens(mol)
    oechem.OETriposAtomNames(mol)
    mols.append(mol)

In [7]:
omegaOpts = OEOmegaOptions()
omega = OEOmega(omegaOpts)

for mol in mols:
    if omega(mol):
        oequacpac.OEAssignCharges(mol, oequacpac.OEAM1BCCELF10Charges())
        conf = mol.GetConf(oechem.OEHasConfIdx(0))
        absFCharge = 0
        sumFCharge = 0
        sumPCharge = 0.0
        for atm in mol.GetAtoms():
            sumFCharge += atm.GetFormalCharge()
            absFCharge += abs(atm.GetFormalCharge())
            sumPCharge += atm.GetPartialCharge()
        print("{}: {} formal charges give total charge {}"
              "; sum of partial charges {:5.4f}".format(mol.GetTitle(), absFCharge,
                                                        sumFCharge, sumPCharge))
        # oechem.OEWriteMolecule(ofs, conf)


: 1 formal charges give total charge -1; sum of partial charges -1.0000
: 0 formal charges give total charge 0; sum of partial charges 0.0000


In [14]:
for atom in mols[0].GetAtoms():
        print(atom, atom.GetPartialCharge())

 0 C -0.16098999977111816
 1 C -0.1662600040435791
 2 C -0.1662600040435791
 3 C -0.10735999792814255
 4 C -0.10735999792814255
 5 C -0.12452999502420425
 6 C 0.9059900045394897
 7 O -0.8342499732971191
 8 O -0.8342499732971191
 9 H 0.09599000215530396
10 H 0.1014999970793724
11 H 0.1014999970793724
12 H 0.14813999831676483
13 H 0.14813999831676483


In [16]:
ofs = oemolostream()
ofs.open('ben-openeye.mol2')
OEWriteMolecule(ofs, mols[0])

0

In [21]:
omega.GetMaxConfs()

200

In [22]:
ofs = oemolostream()
ofs.open('bcd-openeye.mol2')
OEWriteMolecule(ofs, mols[1])

0

In [23]:
mols[1]

<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7f8eb709fe40> >

In [25]:
mols[1].NumConfs()

13

Why are there 13 conformations?